# Project 1: Assembling Genomes


   <div class="alert alert-block alert-danger">
    <center>Due: <b>Tuesday, 8 February, 4:59pm</b>.</center> 
   </div>
   
   <div class="alert alert-block alert-warning">
   <center>
       <b>Collaboration and Resource Policy</b>
    </center>
    For this assignment, you are encouraged to work with one other person. Your team must satisfy the constraints mentioned in <a href="https://www.dropbox.com/s/g6z2xncwc4jsplp/csbio-class2-inked.pdf?dl=0">Class 2</a>.
    
   1. You went to different high schools.
   2. You and your partner have different answers to <em>at least one</em> of these questions:
       - What country were you born in?
       - Have you taken any biology courses at UVA?
       - Have you taken cs3102?
       - Have you taken cs4102?
    
We expect most students will have the best learning experience on this assignment by working with a partner, but if you prefer to work alone it is okay to do this assignment on your own.
    
You are permitted (actually _encouraged_) to discuss these problems with anyone you want, including other students in the class. If you do discuss the specific questions in the assignment with anyone other than your assignment partner and the course staff, though, you should list them in the _External resources used_ section below.
    
You are welcome to use any resources you want for this assignment, other than ones that would defeat the purpose of the assignment. This means you should not look at answers or code from any other students in the class (other than your collaboration with your partner), and if you find code that implements the problem you are being asked to do for the assignment, you should not use that code. You should document all external resource you use that are not part of the course materials in the _External resources used_ section below.

**Team submitting this assignment:**  
<div class="alert alert-success">
    <b><em>Mohit Srivastav (mvs2dq) & Promotto Islam (pti9ea)</em></b>
</div>

**External resources used:** 
<div class="alert alert-success">
<em>It is not necessary to list the course materials, but if you used any other resources, including discussing problems with students not on your team, list them here.</em>
</div>

In this project, we will explore genome assembly — the process of determining the order of nucleotides from fragmented reads that are produced by sequencing machines. 

Genome assembly maps to a very well defined computer science problem, but can get quite complicated, as problems such as full sequence coverage, finding a good length for reads (the $k$ in $k$-mer), and sequencing errors present challenges for sequencing analysis and accuracy. For the required problems (ones everyone is expected to solve), you will be able to assume perfect coverage and no read errors; for the "challenge" problem (that is considered a bonus, and not something we expect everyone to be able to solve, and perhaps may not even be feasible for anyone to solve) you will not be able to rely on such assumptions.

 <div class="alert alert-block alert-warning">
    
<b>Submission</b>: You should work on this assignment by forking the provided Project 1 repository (you probably already did this, following the instructions posted at [https://computingbiology.github.io/project1](https://computingbiology.github.io/project1). Add the other teammate to as a collaborator with write access (also under the Settings tab) to this repository.
    
You should answer the questions and write your code in this Jupyter Notebook. (We don't expect you to need to use any external files or organize your code outside of the notebook, but if you do, make sure to put everything needed to run your code in your repository.) Parts where you are expected to provide and answer (which could be text that can be written in markdown format in the notebook or Python code that runs in the notebook) are marked in green.
        
When you are ready to submit the assignment, you should create a release of the version you are submitting ([github's directions for how to create a release](https://docs.github.com/en/repositories/releasing-projects-on-github/managing-releases-in-a-repository)) tagged as `submit`. After you've done this, send a message in slack to a channel that includes both team members (so the one sending this message should include the other team member) and all of the course staff (`dave`, `Hyun Jae Cho`, and `Anshuman Suri`) and a link to your repository (feel free to add any other useful comments if you want, but the link to your submission repository is sufficient).

## Getting Started

1. Install basic required packages, should be run only once. You may need to restart the jupyter python kernel (under the Kernel menu) after this. (You can execute this directly in the notebook but running the command below.)

In [1]:
%pip install -r requirements.txt

  Created wheel for gvmagic: filename=gvmagic-0.5-py3-none-any.whl size=10379 sha256=9a60a3de110f41a377e54c02399a9fa194a1b4413830bb7a63426c55e5188c01
  Stored in directory: c:\users\bluescreenie_iii\appdata\local\pip\cache\wheels\e5\ad\b2\2ba1aaaf5b2004d96473291c80f98d4524a01c6d6e9d468172
Successfully built gvmagic
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\Bluescreenie_III\AppData\Local\Programs\Python\Python37\pythonw.exe -m pip install --upgrade pip' command.


2. Make sure you have [graphviz](https://graphviz.org/) installed on your system. (On a Mac OS X, `brew install graphviz`. For other platforms, see [_https://graphviz.org/download/_](https://graphviz.org/download/).)

In [2]:
import collections
import matplotlib.pyplot as plt

import utils

## Part 1: Assembling the TeleTubby Genome

For this part, you're given reads generated while trying to sequence the DNA of a mysterious unknown organism with a _tiny_ genome. Some of the course staff is not sure what a TeleTubby is, but I've been assured that no cute creatures were harmed in producing this data (which was generated synthetically).

By answering the following questions, you will learn how to assemble the original genome sequence from sequence reads.

Sequencing data is often stored in the FASTQ file format, with is a simple ASCII format that is somewhat human-readable. 

In _TeleTubby.fastq_ (which is included in the repository you forked), you will find the data that was read from the TeleTubby genome. Each four lines of the file repeat the same pattern. For example,
```
@TeleTubby Genome: Project 1
TAAAATGG
+
HAICDF5I
```
The first line contains the metadata that encodes the name of the read, the experiment type, the kind of sequencing machine used, etc. 

The second line is the sequence of bases that was read. 

The third line is just a placeholder.

The fourth line is a sequence of base qualities that encodes the qualities for the corresponding bases in the sequence line. We will discuss and use this for Problem 2.

Read in the data from `TeleTubby.fastq`:

In [3]:
# Read sequence reads (error-free) from file
sequence_reads, qualities = utils.read_fastq('TeleTubby.fastq')

In [4]:
sequence_reads

['ACCACGTA',
 'CCACCACG',
 'CGATGGTA',
 'TTTTGGTC',
 'ACTCGGAG',
 'CGGAGGAT',
 'ATAGGTGA',
 'TAAAATGG',
 'TATGCCAA',
 'TACAGGTG',
 'CAACGCTT',
 'TGACTCGG',
 'AGGTGTCG',
 'ACGTATCC',
 'TGGTCGTA',
 'CCAACGCT',
 'TGTTGATA',
 'CTCGGAGG',
 'TGATAAAA',
 'GGCCACAT',
 'ATAATTTT',
 'ATGGTAAA',
 'GTAAACCT',
 'ACAGGTGT',
 'ATCCCCCG',
 'ATAAAATG',
 'AATGGCCA',
 'AGGTGACT',
 'GATGGTAA',
 'GCACATGG',
 'GGATCACT',
 'TGATAGGT',
 'CACCACGT',
 'ATCACTTT',
 'CGTACATC',
 'TTAACGAC',
 'TCGTGTTA',
 'TCGTACAT',
 'CACATGGC',
 'CTCCGGTA',
 'AGTGATAG',
 'TCATGCAC',
 'TGTCATGC',
 'CACATAAT',
 'ACGCTTAC',
 'TTGATAAA',
 'CCCGCGCA',
 'CCACGTAT',
 'GTCGATGG',
 'GCATCGTG',
 'ACTTTAAC',
 'TACCGACG',
 'TCGAGTGA',
 'AGGGGATT',
 'TCGTACAG',
 'GGAGGATC',
 'CACGTATC',
 'CACTTTAA',
 'CTACCGAC',
 'CCGCGCAT',
 'TACATCCC',
 'CCACATAA',
 'CTTTAACG',
 'ACGACCAC',
 'CGAGTGAT',
 'ATAGGGGA',
 'GGTCGTAC',
 'CATCGTGT',
 'TGTGTCGA',
 'ATAAAATG',
 'GTGTCGAT',
 'GTCGTACA',
 'CGCGCATC',
 'CCCCACCA',
 'CCACGGAC',
 'GGTAAACC',
 'TGGCCACA',

### Melting Temperature

The GC-content (or the ratio of G and C nucleotides) is related to the melting temperature of a DNA double helix structure. 

The following equation can be used to estimate the melting temperature (in degrees Celsius) of DNA for a particular sequence:

\begin{equation*}
t_m = 64.9+0.41(\%GC)-\frac{500}{\text{length of sequence}}
\end{equation*}

As a reference, the human genome is known to have between 35%-60% GC-content. 

<div class="alert alert-success">

**Problem 1.** Calculate the melting temperature for the TeleTubby genome using the formula above. Assume that the sequence is 200 nucleotides long, and that the provided read data has equal coverage everywhere so the G-C distribution in the reads is very close to the actual G-C distribution for the genome. 
    </div>

In [5]:
def GC_percent(sequence_reads):
    tot_acid = 0
    tot_GC = 0
    for sequence in sequence_reads:
        for acid in sequence:
            if acid == 'C' or acid == 'G':
                tot_GC += 1
            tot_acid += 1
    return tot_GC/tot_acid*100, tot_acid

In [6]:
gc, seq_len = GC_percent(sequence_reads)
print("The melting temperature is:", 64.9 + 0.41*gc - (500/seq_len), "degrees C")

The melting temperature is: 85.02882653061225 degrees C


## Interpreting Read Quality

Phred33 quality scores are represented as the character with an ASCII code equal to its value + 33 (to make them easy to print alongside genome sequences). The Phred scores $Q$ are related to error probabilities $P$ as: $Q = -10 \log_{10}(P)$. The table linked here provides a mapping from the Phred values provided in a fastq file and the $Q$ values: [Quality Score Encoding](https://support.illumina.com/help/BaseSpace_OLH_009008/Content/Source/Informatics/BS/QualityScoreEncoding_swBS.htm) 

<div class="alert alert-success">
    
**Problem 2.** Using the Phred scores, identify the _least likely to be correct_ read in the TeleTubby.fastq data, which is the read with the lowest probability that it contains _no_ errors.

</div>

In [7]:
# Write your code for Problem 2 here. 
# It should output the read (nucleotide sequence) with the lowest probability of containing no errors.

def read_quality(qualities):
    lowest_prob_sum = 1
    index = 0
    lowest_index = 0
    for q in qualities:
        prob_sum = 0
        for letter in q:
            #qscore + 33 = ASCII 
            #P =  10^(-q/10)
            qscore = ord(letter)-33
            prob_sum+=(10**(-qscore/10))
        # print(prob_sum)
        if prob_sum < lowest_prob_sum:
            lowest_prob_sum = prob_sum
            lowest_index = index
        index+=1
    return lowest_prob_sum,lowest_index
prob,index = read_quality(qualities)
# print(read_quality(qualities))
# print(qualities)
print('The seqence',sequence_reads[index],'has the lowest probability that it contains no errors with a probability of',prob,'.')
    

The seqence CCATCGTA has the lowest probability that it contains no errors with a probability of 0.001307810616351377 .


Note: For the following problems, you can ignore the read quality scores, and safely assume there are no actual errors in the provided TeleTubby reads.

## Frequency analysis

Looking at repetitions in the sequence can be helpful in estimating the "redudancy" in the organisms. Eucaryote genomes have a lot of redundancy, while smaller organisms like bacteria have highly packed genomes. One heuristic to estimate this before actually performing the assembly could be looking at how often certain $k$-mers are repeated.

<div class="alert alert-success">
    
**Problem 3.**
    Print out the 3 most frequent <em>k</em>-mers in the TeleTubby reads with their frequencies. (As in Problem 1, doing this from the reads only produces the same result as from the genome if the read coverage is equal everywhere, but it should be a close estimate.)
</div>

In [8]:
# Your code here
def freq_analysis(k,sequence_reads):
#     print(sequence_reads)
    if k <= len(sequence_reads[0]):
        all_kmer = {}
        for read in sequence_reads:
            i = 0
            while i+k <= len(read):
                kmer = read[i:i+k]
                if kmer in all_kmer:
                    all_kmer[kmer]+=1
                else:
                    all_kmer[kmer]=1
                i+=1
        return(all_kmer)
    else:
        return('k too large')
total_kmer_in_seq = freq_analysis(len(sequence_reads[0]),sequence_reads) #Change first parameter as needed (also changed k due to email chain in discussion)
try:
    sorted_kmer = sorted(total_kmer_in_seq.items(), key = lambda x:x[1])
#     print(sorted_kmer)
    print('Most frequent k-mers in sequence:')
    print(sorted_kmer[-1])
    print(sorted_kmer[-2])
    print(sorted_kmer[-3])
except:
    print('Change value of k')

Most frequent k-mers in sequence:
('TAATTTTT', 2)
('GTGATAGG', 2)
('ATAAAATG', 2)


## Greedy Assembly

Given a set of sequence fragments, the objective of assembly can be viewed as finding the shortest sequence that contains all the fragments.

One of the approaches to assemble the genome from the given reads is a greedy algorithm:

```
while (len(fragments) > 1):
   calculate pairwise alignments of all pairs of fragments
   merge the two fragments with the largest overlap
genome = fragment[0] # the single remaining fragment is the genome
```


<div class="alert alert-success">
    
**Problem 4 (a)**. What would the runtime be of this algorithm, given $n$ $k$-mer reads? (If you are not sure how to express your answer to this question, review [_Cost of Computation_](https://computingbiology.github.io/complexity/).)
    </div>

<i>Answer</i>: O(n log(n)). This is because calculating the pairwise alignments takes n time, while the combinining of the fragments takes log(n) runs (since you are doubling the size of the fragments each time). Hence the runtime.

<div class="alert alert-success">

**Problem 4 (b)**. Is this algorithm guaranteed to find the correct genome? (A good answer will define what _correct_ means here, and explain why the algorithm is or is not guaranteed to find it.)

<i>Answer</i>:

## Graph-based Assembly

Graphs for genome assembly can be constructed in two ways:

- de Bruijn graph: Processing $k-$mers as nodes, with $(k-1)-$mers as edges, and
- Overlap graph: Processing $k-$mers as edges, with $(k-1)-$mers as nodes.

A de Bruijn graph can be processed to find Euler paths, while Overlap graphs can be processed to find Hamiltonian paths. Both of these methods can be used reconstruct the original genome.

<div class="alert alert-success">
    
**Problem 5.**  Use one of these two techniques to reconstruct the TeleTubby genome from the provided sequence reads. 
    
</div>
    
We have provided some template code below that may be helpful (but feel free to ignore this is you prefer).

In [9]:
# # Read reads into graph
# # "Each read is a 8-mer"
# def build_graph(k_mers):
#     edges = []
#     nodes = set()
#     # k = len(k_mers[0])
#     # print(k_mers)
#     for read in k_mers:
#         # print(read)
#         nodes.add(read[:-1])
#         nodes.add(read[1:])
#         edges.append([read[:-1],read[1:]])
#         # print(nodes)
#         # break
#     # print(nodes)
#     # Your code here
#     # print(edges)
#     return nodes, edges

In [10]:
def Euler_Check(nodes, edges):
    """Function that takes in the nodes and edges, 
    then spits out whether an Eulerian path is available
    and a revised set of edges if you need to connect two ends to help make cycles
    """
    nodes2 = dict.fromkeys(nodes, 0) #initializes a dictionary with in - out as the value
    for edge in edges:
        nodes2[edge[0]] -= 1 #edge goes out node
        nodes2[edge[1]] += 1 #edge goes in node
#         print("edge", edge[2],"goes out of node", edge[0],"and into node",edge[1])
#         print(nodes2)
    
    connection = [None, None, None] #this connects two nodes that are at the ends
    connectionMade = [False, False] #checks if either connectors were created
    for node in nodes2:
        if nodes2[node] == 1 and not connectionMade[0]:
            connection[0] = node
            connectionMade[0] = True
            
        elif nodes2[node] == -1 and not connectionMade[1]:
            connection[1] = node
            connectionMade[1] = True
            
        elif nodes2[node] != 0 and connectionMade[0] and connectionMade[1]:
            return False, None
            
    if connection[0] != None and connection[1] != None:
        connection[2] = connection[0] + connection[1][1:]
    
    edges2 = edges #just so you don't modify edges from before
    if connection[0] != None and connection[1] != None:
        edges2 = edges + [tuple(connection)] #adds a connector

    return True, edges2

In [11]:
def build_graph(k, sequence_reads):
    """takes in the sequence reads and then compiles all the nodes and edges
    also uses the Euler_Check function to add any necessary edges and check if the graph is Eulerian"""
    if k > len(sequence_reads[0]):
        print("Pick a smaller k!")
        return None, None
    edges = []
    nodes = set()
    for read in sequence_reads:
        i = 0
        while i+k <= len(read):
            tempNode1 = read[i:i+k - 1] #first node of (k-1) mer
            tempNode2 = read[i+1:i+k] #second node of (k-1) mer

            edges.append((tempNode1, tempNode2, read[i:i+k])) #edges are tuples of node1, node2, and value
            nodes.add(tempNode1) 
            nodes.add(tempNode2)
            i += 1
    check, edges = Euler_Check(nodes, edges)
    if check:
        print("Eulerian Graph Found!")
        return list(nodes), edges
    else:
        print("Graph not Eulerian!")
        return None, None

In [12]:
# nodes, edges = build_graph(2,sequence_reads)
nodes, edges = build_graph(3, ['AAT','ATG','ATG','ATG','CAT','CCA','GAT','GCC','GGA','GGG','GTT','TAA','TGC','TGG','TGT']) #example from reading
# nodes, edges = build_graph(3, ['ATGCTA', 'GCTAGC', 'TAGCAC', 'GCACAT', 'ACATGC'])

Eulerian Graph Found!


In [13]:
print(("n1", "n2", "edge"))
print("-"*20)
[print(edge) for edge in edges]
print("nodes:")
print(nodes)

('n1', 'n2', 'edge')
--------------------
('AA', 'AT', 'AAT')
('AT', 'TG', 'ATG')
('AT', 'TG', 'ATG')
('AT', 'TG', 'ATG')
('CA', 'AT', 'CAT')
('CC', 'CA', 'CCA')
('GA', 'AT', 'GAT')
('GC', 'CC', 'GCC')
('GG', 'GA', 'GGA')
('GG', 'GG', 'GGG')
('GT', 'TT', 'GTT')
('TA', 'AA', 'TAA')
('TG', 'GC', 'TGC')
('TG', 'GG', 'TGG')
('TG', 'GT', 'TGT')
('TT', 'TA', 'TTA')
nodes:
['AA', 'GC', 'TT', 'CA', 'GG', 'AT', 'TA', 'GA', 'TG', 'GT', 'CC']


In [14]:
# Needed to properly visualize graphs
%load_ext gvmagic

In [15]:
# Helper function to visualize graphs
%dotstr utils.viz_graph(nodes, [edge[0:2] for edge in edges]) #has the nodes and the edges defined by the two nodes

FileNotFoundError: [WinError 2] The system cannot find the file specified

In [16]:
edgeDict = {} #generates a dictionary of edges from the original list
for edge in edges:
    if edge[:2] not in edgeDict:
        edgeDict[edge[:2]] = [edge[2]]
    else:
        edgeDict[edge[:2]] += [edge[2]]

def Single_Cycle(node, edgeDict, edges):
    
    startNode = node #this is where you start the cycle
    curNode = node #currently at the start
    cycleStart = True #just a measure to make sure the while loop runs
        
    cycle = [curNode] #start the cycle at the starting node
    while curNode != startNode or cycleStart:
        cycleStart = False #the cycle is not set to start
        for edge in edges:
            if edge[0] == curNode and edgeDict[edge[:2]] != []: #finds an appropriate edge to move in
                curNode = edge[1] #moves to the next node
                edgeDict[edge[:2]] = edgeDict[edge[:2]][:-1] #removes that edge from the dictionary by removing a list value
                cycle.append(curNode)
                break #restart the process!
                
    return cycle[:-1] #return the path of everything but remove the last node so it isn't a cycle

def Euler_Cycle(nodes, edgeDict, edges):
    starting_cycle = Single_Cycle(nodes[0], edgeDict, edges) #start at the beginning of the list cause why not
    print("cycle starting at", nodes[0] +":", starting_cycle)
    while any(list(edgeDict.values())): #checks if there are any paths that haven't been eliminated
        new_starting_node = None
        for edge in edges:
            if edgeDict[edge[:2]] != [] and edge[0] in starting_cycle: #find new edge in the starting cycle
                new_starting_node = edge[0] #start the new cycle here and break the loop
                break
        
        cycle_prime = Single_Cycle(new_starting_node, edgeDict, edges)
        print("new cycle starting at", new_starting_node+":", cycle_prime)
        
        insertion_index = starting_cycle.index(new_starting_node)
        
        for n,i in enumerate(cycle_prime):
            starting_cycle.insert(insertion_index + n, i)
        
        print("new updated cycle:", starting_cycle)
    
    return starting_cycle

EulerianCycle = Euler_Cycle(sorted(nodes), edgeDict, edges)

cycle starting at AA: ['AA', 'AT', 'TG', 'GC', 'CC', 'CA', 'AT', 'TG', 'GG', 'GA', 'AT', 'TG', 'GT', 'TT', 'TA']
new cycle starting at GG: ['GG']
new updated cycle: ['AA', 'AT', 'TG', 'GC', 'CC', 'CA', 'AT', 'TG', 'GG', 'GG', 'GA', 'AT', 'TG', 'GT', 'TT', 'TA']


In [35]:
def assemble_sequence(nodes, edges):
    assembled_sequence = ""
    # Your code here
    return assembled_sequence

In [ ]:
# Output assembled sequence
# Hint: Sequence is 200 nucleotides long

assmebled_seq = assemble_sequence(nodes, edges)

<div class="alert alert-success"> 
    
**Problem 6.** Which of the two assembly methods did you use for problem 5, and why? (Hint: consider how the costs scale with the number of reads.)
    
</div>

## Part 2: Sequencing SARS-CoV-2 virus

Let's move on from TeleTubbies to real-world organisms. For this problem, you'll assemble a genome for a variant of the SARS-CoV-2 virus. You're given reads from <i>actual</i> genome sequencing runs in the provided `SARS-CoV2.fastq` file. The file is based on the first Covid genome that was submitted on 5 January 2020: [https://www.ncbi.nlm.nih.gov/nuccore/NC_045512](https://www.ncbi.nlm.nih.gov/nuccore/NC_045512) But, you should be reconstructing a sequence from the provided reads without looking at the source sequence.

<div class="alert alert-success">
    
**Problem 7.**  Reconstruct the SATS-CoV2 genome from the provided sequence reads using $k=25$. 
    
</div>

You can re-use your implementation from Problem 5 and simply run it on the new data (depending on what you did for Problem 5, that might be enough to solve this problem). 

Print out your reconstructed sequence to a file `output.txt` (and add this file to the repo you submit). For this part, we will still assume that all the reads are error-free. 

In [ ]:
# Read sequence reads
sequence_reads_covid, qualities_covid = utils.read_fastq('SARS-CoV2.fastq')

In [ ]:
# Read reads into graph
nodes_covid, edges_covid = build_graph(sequence_reads_covid)

In [ ]:
# Call main assembly algorithm
assmebled_covid_seq = assemble_sequence(nodes_covid, edges_covid)

In [ ]:
# Write assembled sequence to file
with open("output.txt", "w") as f:
    f.write(assmebled_covid_seq)

# Part 3: Error-Aware Assembly (Challenge Problem)

<div class="alert alert-warning">
    This problem is a "Challenge Problem". This means it is a problem of unknown difficulty that might be quite challenging (unlike the earlier problems, we don't have a reference solution for this one, or a clear idea how hard it might be). We do hope all students will at least attempt this and that more ambitious students will work hard to solve it and learn interesting things by the attempt (whether or not it is successful), but not get frustrated if you can't get to the desired answer.  As a "Challenge Problem" it means that you shouldn't be worried if you are not able to solve this, though, and you can get full expected credit on this assignment without answering it.
</div>


In the parts above, we assumed error-free reads while assembling $k$-mers. As much as we'd like that, actual reads can (and do) have errors, captured by their Phred scores. 

For this question, you're given raw, actual reads from sequencing runs. Download the reads from this file:
https://sra-pub-sars-cov2.s3.amazonaws.com/sra-src/SRR11528307/ABS2-LN-R1_cleaned_paired.fastq.gz.  


<div class="alert alert-success">
    
**Problem 8 (Challenge).** Give the reads in the linked fastq file above, including their Phred33 quality score, assemble the most likely genome. Your solution should output the assembled sequence in `challenge.txt`. Provide a brief explantion of how your algorithm works and interesting things you learned in developing it.
    
</div>
    
This is an open-ended question. You are free to use any approach to deal with the issue. Make sure you provide your code, along with any assumptions you may have.

_Write a description of your algorithm, and things you learned from working on this here._

In [ ]:
# implementation 

   <div class="alert alert-block alert-danger">
    <center>
        
 **Remember to follow the submission directions above to submit your assignment by 4:59pm on Tuesday, 8 February.**
    
 </center>
 </div>